In [2]:
! pip install matplotlib
! pip install qiskit


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\janko\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\janko\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import CircuitInstruction, Instruction, Qubit
from qiskit.utils import QuantumInstance
from qiskit.visualization import plot_histogram

In [3]:
qasmStr = """OPENQASM 2.0; include "qelib1.inc"; qreg q[3]; creg c[3]; rz(0.54871997) q[2]; u3(1.1865947,2.4192026,1.1759944) q[1]; rx(4.4595309) q[0]; crz(1.5542926) q[2],q[0]; z q[1]; ccx q[1],q[0],q[2]; cswap q[2],q[0],q[1]; ry(2.5450405) q[0]; y q[1]; t q[2]; sdg q[2]; t q[1]; s q[0]; ccx q[1],q[2],q[0]; cu3(1.9503811,4.5063394,2.2821983) q[2],q[0]; h q[1]; cswap q[0],q[1],q[2]; cz q[1],q[0]; x q[2]; cy q[0],q[1]; z q[2]; cu3(5.469467,2.3869679,0.34975866) q[2],q[0]; u3(1.4534986,3.8510188,2.4941133) q[1]; rzz(1.3417746) q[0],q[2]; sdg q[1]; measure q[0] -> c[0]; measure q[1] -> c[1]; measure q[2] -> c[2]; """
circuit = QuantumCircuit.from_qasm_str(qasmStr)
# circuit.draw(output='text')

In [4]:
# circuit.qasm()

In [5]:
import random

def prettyfy(str: str):
    return str.replace("\n", " ")

def add_double_negation(circuit: QuantumCircuit):
    qubits = circuit.qubits
    instructions = circuit.data
    rand_qubit = qubits[0]

    x_gate = CircuitInstruction(
            operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=()),
            qubits=(rand_qubit,),
            clbits=()
        )
    
    new_circuit = QuantumCircuit(circuit.qubits + circuit.clbits)
    x_position = random.randint(0, len(instructions))
    for idx, instruction in enumerate(instructions):
        if(idx == x_position):
            new_circuit.append(x_gate)
            new_circuit.append(x_gate.copy())
        new_circuit.append(instruction)
    return new_circuit

""" Y = -XYX
"""
def change_y_xyx(circuit: QuantumCircuit):
    qubits = circuit.qubits
    instructions = circuit.data

    # for every qubit collect instructions where it is mentioned the correct order
    perQubit = {q: [ins for ins in instructions if q in ins.qubits] for q in qubits}
    print(perQubit)
    expectedNames = ["rx"]
    for q in qubits:
        perQubit[q]
    pass

# circ = add_double_negation(circuit)
# change_y_xyx(circuit)
# circ.draw(output="text")

In [6]:
# qasmStr

In [7]:
# prettyfy(circ.qasm()) # order changes :(

In [8]:
from qiskit.qasm import Qasm
from qiskit.qasm.node import *

def gen_qasm(self):
    if isinstance(self, Barrier):
        return "barrier " + gen_qasm(self.children[0]) + ";"
    elif isinstance(self, BinaryOp):
        return (
            "(" + gen_qasm(self.children[1]) + self.children[0].value + gen_qasm(self.children[2]) + ")"
        )
    elif isinstance(self, BinaryOperator):
        return self.value
    elif isinstance(self, Cnot):
        return "CX " + gen_qasm(self.children[0]) + "," + gen_qasm(self.children[1]) + ";"
    elif isinstance(self, Creg):
        return "creg " + gen_qasm(self.id) + ";"
    elif isinstance(self, CustomUnitary):
        string = self.name
        if self.arguments is not None:
            string += "(" + gen_qasm(self.arguments) + ")"
        string += " " + gen_qasm(self.bitlist) + ";"
        return string
    elif isinstance(self, ExpressionList):
        return ",".join([gen_qasm(self.children[j]) for j in range(self.size())])
    elif isinstance(self, External):
        return gen_qasm(self.children[0]) + "(" + gen_qasm(self.children[1]) + ")"
    elif isinstance(self, Format):
        return f"{self.language} {self.version()};"
    elif isinstance(self, Gate):
        string = "gate " + self.name
        if self.arguments is not None:
            string += "(" + gen_qasm(self.arguments) + ")"
        string += " " + gen_qasm(self.bitlist) + "\n"
        string += "{\n" + gen_qasm(self.body) + "}"
        return string
    elif isinstance(self, GateBody):
        string = ""
        for children in self.children:
            string += "  " + gen_qasm(children) + "\n"
        return string
    elif isinstance(self, Id):
        return self.name
    elif isinstance(self, IdList):
        return ",".join([gen_qasm(self.children[j]) for j in range(self.size())])
    elif isinstance(self, If):
        return (
            "if("
            + gen_qasm(self.children[0])
            + "=="
            + str(self.children[1].value)
            + ") "
            + gen_qasm(self.children[2])
        )
    elif isinstance(self, IndexedId):
        return self.name + "[%d]" % self.index
    elif isinstance(self, Int):
        return "%d" % self.value
    elif isinstance(self, Measure):
        return "measure " + gen_qasm(self.children[0]) + " -> " + gen_qasm(self.children[1]) + ";"
    elif isinstance(self, Opaque):
        string = "opaque %s" % self.name
        if self.arguments is not None:
            string += "(" + gen_qasm(self.arguments) + ")"
        string += " " + gen_qasm(self.bitlist) + ";"
        return string
    elif isinstance(self, Prefix):
        return self.children[0].value + "(" + gen_qasm(self.children[1]) + ")"
    elif isinstance(self, PrimaryList):
        return ",".join([gen_qasm(self.children[j]) for j in range(self.size())])
    elif isinstance(self, Program):
        string = ""
        for children in self.children:
            string += gen_qasm(children) + "\n"
        return string
    elif isinstance(self, Qreg):
        return "qreg " + gen_qasm(self.id) + ";"
    elif isinstance(self, Real):
        return str(float(self.value))
    elif isinstance(self, Reset):
        return "reset " + gen_qasm(self.children[0]) + ";"
    elif isinstance(self, UnaryOperator):
        return self.value
    elif isinstance(self, UniversalUnitary):
        return "U(" + gen_qasm(self.children[0]) + ") " + gen_qasm(self.children[1]) + ";"

In [9]:
from qiskit.qasm import Qasm
from qiskit.qasm.node import *

# Hack to remove 'include "qelib1.inc"' gates
# modifies the original programNode
def remove_qelib_inc(programNode):
    del programNode.children[1:43]
    return programNode

def readd_include(qasmCode):
    return qasmCode.replace("OPENQASM 2.0;", "OPENQASM 2.0; include \"qelib1.inc\";")

def renameTransformation(programNode):
    programNode = remove_qelib_inc(programNode)
    
    def uniqueIds(node):
        idsFound = []
        for child in node.children:
            if isinstance(child, Creg) or isinstance(child, Qreg):
                indexedIdNode = child.children[0]
                idNode = indexedIdNode.children[0]
                idsFound += [idNode.name]
            else:
                idsFound += uniqueIds(child)
        return idsFound

    # mapping : srcName -> destName
    def renames(node, mapping):
        for child in node.children:
            if isinstance(child, Id):
                if child.name in mapping:
                    child.name = mapping[child.name]
            # indexed id has its own name field,
            # which messes with the generation of IndexedId
            elif isinstance(child, IndexedId): 
                if child.name in mapping:
                    child.name = mapping[child.name]
                renames(child, mapping)
            else:
                renames(child, mapping)
        
    ids = uniqueIds(programNode)
    mapping = {}
    for i in range(1, len(ids), 2):
        mapping[ids[i]] = ids[i-1]
        mapping[ids[i-1]] = ids[i]
    
    renames(programNode, mapping)
    return programNode

# a = Qasm(data=qasmStr)
# print(a.parse().children[50])
# print(a.parse().qasm())


# print(qasmStr)
# print(readd_include(prettyfy(gen_qasm(remove_qelib_inc(a.parse())))))
# print(readd_include(prettyfy(gen_qasm(renameTransformation(a.parse())))))

In [12]:
# may break on custom gates (Gate, Opaque, GateBody)
def swapTransformation(programNode):
    programNode = remove_qelib_inc(programNode)

    independent_sets = []
    current_independent = []

    def get_children_ids(node):
        ids = [] # coded in str (c, -1) for all, (c, 0) for int
        if isinstance(node, IndexedId):
            idName = node.children[0].name
            intNum = node.children[1].value
            ids.append((idName, intNum))
            return ids
        elif isinstance(node, Id):
            idName = node.name
            intNum = -1
            ids.append((idName, intNum))
            return ids
        for child in node.children:
            if isinstance(child, IndexedId):
                idName = child.children[0].name
                intNum = child.children[1].value
                ids.append((idName, intNum))
            elif isinstance(child, Id):
                idName = child.name
                intNum = -1
                ids.append((idName, intNum))
            else:
                ids += get_children_ids(child)
        return ids
    
    def handle_dependence(node):
        nonlocal current_independent
        if isinstance(node, CustomUnitary): #quickfix
            if len(node.children) == 3:
                ids = get_children_ids(node.children[2])
            else:
                ids = get_children_ids(node.children[1])
        else:
            ids = get_children_ids(node)
        new_part = (node, ids)
        is_dependent = False
        for cur_node in current_independent:
            for new_id in ids:
                for cur_id in cur_node[1]:
                    if (new_id[1] == -1 and cur_id[0] == new_id[0]) or new_id == cur_id:
                        is_dependent = True
        if(is_dependent):
            independent_sets.append(current_independent)
            current_independent = [new_part]
        else:
            current_independent.append(new_part)

    for child in programNode.children:
        if isinstance(child, Barrier):
            independent_sets += [current_independent, [(child, ())]]
            current_independent = []
        elif isinstance(child, Cnot):
            handle_dependence(child)
        elif isinstance(child, Creg): #TODO
            independent_sets += [current_independent, [(child, ())]]
            current_independent = []
        elif isinstance(child, CustomUnitary):
            handle_dependence(child)
        # elif isinstance(child, External):
        #     handle_dependence(child)
        elif isinstance(child, If): # unsure of scope here (are you able to define a qreg/creg in a body?)
            handle_dependence(child)
        elif isinstance(child, Measure):
            independent_sets += [current_independent, [(child, ())]]
            current_independent = []
        elif isinstance(child, Qreg):
            independent_sets += [current_independent, [(child, ())]]
            current_independent = []
        elif isinstance(child, Reset): # handle id and indexed_id
            handle_dependence(child)
        elif isinstance(child, UniversalUnitary): # handle id and indexed_id
            handle_dependence(child)
        elif isinstance(child, Format):
            independent_sets += [current_independent, [(child, [()])]]
            current_independent = []
        elif isinstance(child, Gate):
            independent_sets += [current_independent, [(child, [()])]]
            current_independent = []
        else: # Throw Error - will error out custom gates
            print("ERROR: ", child)
    
    independent_sets += [current_independent]
    
    new_children = []
    for op_list in independent_sets:
        new_children.extend([i[0] for i in reversed(op_list)])
    new_program = Program(new_children)
        
    return new_program

a = Qasm(data=qasmStr)

# print(gen_qasm(a.parse()))
# print()
# print(swapTransformation(a.parse()).children)
# print()
print("original QASM String:")
print(qasmStr)
print()
print("transformed QASM String:")
print(readd_include(prettyfy(gen_qasm(swapTransformation(a.parse())))))


original QASM String:
OPENQASM 2.0; include "qelib1.inc"; qreg q[3]; creg c[3]; rz(0.54871997) q[2]; u3(1.1865947,2.4192026,1.1759944) q[1]; rx(4.4595309) q[0]; crz(1.5542926) q[2],q[0]; z q[1]; ccx q[1],q[0],q[2]; cswap q[2],q[0],q[1]; ry(2.5450405) q[0]; y q[1]; t q[2]; sdg q[2]; t q[1]; s q[0]; ccx q[1],q[2],q[0]; cu3(1.9503811,4.5063394,2.2821983) q[2],q[0]; h q[1]; cswap q[0],q[1],q[2]; cz q[1],q[0]; x q[2]; cy q[0],q[1]; z q[2]; cu3(5.469467,2.3869679,0.34975866) q[2],q[0]; u3(1.4534986,3.8510188,2.4941133) q[1]; rzz(1.3417746) q[0],q[2]; sdg q[1]; measure q[0] -> c[0]; measure q[1] -> c[1]; measure q[2] -> c[2]; 

transformed QASM String:
OPENQASM 2.0; include "qelib1.inc"; qreg q[3]; creg c[3]; rx(4.4595309) q[0]; u3(1.1865947,2.4192026,1.1759944) q[1]; rz(0.54871997) q[2]; z q[1]; crz(1.5542926) q[2],q[0]; ccx q[1],q[0],q[2]; cswap q[2],q[0],q[1]; t q[2]; y q[1]; ry(2.5450405) q[0]; s q[0]; t q[1]; sdg q[2]; ccx q[1],q[2],q[0]; h q[1]; cu3(1.9503811,4.5063394,2.2821983) q[2],q